In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# data import 
data = pd.read_csv("", sep = ";")
order = data['Unnamed: 1']
data = data.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1)
data = data.replace('  ', '0').fillna('0').astype('string')
pd.set_option('display.min_rows', 1000)

In [ ]:
data = pd.read_csv("")
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
def encode_arr(arr):
    encoded_arr = []
    for i in range(len(arr)):
        encoded_arr.append(Card(arr[i]))
    return encoded_arr

In [ ]:
def create_player_df(df, s, e):    
    arr_player = []
    for i in range(1, len(df)):
        player_hands = list(df.loc[i])[s:e]
        prev_row = list(df.loc[i - 1])
        unique_arr = encode_arr(list(set(player_hands + prev_row)))
        arr_player.append(unique_arr)
    return arr_player
        

In [ ]:
df_player1 = create_player_df(data, 0, 2)
df_player2 = create_player_df(data, 2, 4)
df_player3 = create_player_df(data, 4, 6)
df_player4 = create_player_df(data, 6, 8)
df_player5 = create_player_df(data, 8, 10)
df_player6 = create_player_df(data, 10, 12)

In [ ]:
games_indexes = []
for i in range(1, len(data)):
    if data.iloc[i, 0] != data.iloc[i - 1, 0]:
        games_indexes.append(i)

# CLASSES

CLASS: create card with value and suit 

In [ ]:
class Card:
  def __init__(self, card):
    if card != '0':
      self.value = card[0] #value 
      self.suit = card[1] #suit
    else:
      self.value = '0'
      self.suit = '0'

CLASS: create player with values and suits counts

In [ ]:
class Player:
    def __init__(self, values_dict, suits_dict):
        self.values = values_dict
        self.suits = suits_dict

CLASS: create combination with name, score, bonus info, kicker and player's name

In [ ]:
class Combination:
    def __init__(self, comb_name, comb_score, score, bonus_info):
        self.combination_name = comb_name
        self.combination_score = comb_score
        self.score = score
        self.bonus = bonus_info
    def append_kicker(self, kicker):
        self.kicker = kicker
    def append_player(self, player):
        self.player = player

# FUNCTIONS

FUNCTION: encode dataframe to 2D array of Card objects

In [ ]:
def encode_df_cards(df_tables):
  tables = []
  for i in range(len(df_tables)):
    cards = []
    for j in range(df_tables.shape[1]):
      cards.append(Card(df_tables.iloc[i, j]))
    tables.append(cards)
  return tables

FUNCTION: return sorted scores of each value in string 

In [ ]:
def get_value_score(string):
    scores = []
    sorted_arr = list(string)
    if '2' in sorted_arr:
        scores.append(2)
    if '3' in sorted_arr:
        scores.append(3)
    if '4' in sorted_arr:
        scores.append(4)
    if '5' in sorted_arr:
        scores.append(5)
    if '6' in sorted_arr:
        scores.append(6)
    if '7' in sorted_arr:
        scores.append(7)
    if '8' in sorted_arr:
        scores.append(8)
    if '9' in sorted_arr:
        scores.append(9)
    if 'T' in sorted_arr:
        scores.append(10)
    if 'J' in sorted_arr:
        scores.append(11)
    if 'Q' in sorted_arr:
        scores.append(12)
    if 'K' in sorted_arr:
        scores.append(13)
    if 'A' in sorted_arr:
        scores.append(14)
    if '0' in sorted_arr:
        scores.append(0)
    return(sorted(scores))

FUNCTION: return straight score

In [ ]:
def get_straight_score(straight_set):
    alphabet = "23456789TJQKA"
    order = dict(zip(alphabet, range(len(alphabet))))
    sorted_arr = sorted(straight_set, key = lambda word: [order[c] for c in word])
    string = ''.join(sorted_arr)

    if "TJQKA" in string:
        score = 14
    elif "9TJQK" in string:
        score = 13
    elif "89TJQ" in string:
        score = 12
    elif "789TJ" in string:
        score = 11
    elif "6789T" in string:
        score = 10
    elif "56789" in string:
        score = 9
    elif "45678" in string:
        score = 8
    elif "34567" in string:
        score = 7
    elif "23456" in string:
        score = 6
    elif "A2345" in string:
        score = 5
    else:
        score = 0
    return score

In [ ]:
def get_straight_scores(straight_set):
    scores = []
    alphabet = "23456789TJQKA"
    order = dict(zip(alphabet, range(len(alphabet))))
    sorted_arr = sorted(straight_set, key = lambda word: [order[c] for c in word])
    string = ''.join(sorted_arr)

    if "TJQKA" in string:
        scores.append(14)
    if "9TJQK" in string:
        scores.append(13)
    if "89TJQ" in string:
        scores.append(12)
    if "789TJ" in string:
        scores.append(11)
    if "6789T" in string:
        scores.append(10)
    if "56789" in string:
        scores.append(9)
    if "45678" in string:
        scores.append(8)
    if "34567" in string:
        scores.append(7)
    if "23456" in string:
        scores.append(6)
    if "A2345" in string:
        scores.append(5)
    return scores

FUNCTION: return dictionaries with values and suits counts of community cards

In [ ]:
def get_community_values_suits_dict(community_cards):
    community_values = {} # dictionary of value counts for community cards for one row
    community_suits = {'S' : '', 'C' : '', 'H' : '', 'D' : '', 'pS' : '', 'pC' : '', 'pH' : '', 'pD' : ''} # dictionary of suits counts for community cards
    for i in range(len(community_cards)):
        card = community_cards[i] # take object card
        if card.value != '0':
            # count values
            if card.value in community_values:
                community_values[card.value] += 1
            else:
                community_values[card.value] = 1
            # count suits     
            if (card.suit) in community_suits:
                community_suits[card.suit] += card.value
            # community suits = {'K' : 3, '9' : 2}
            # community suits = {'S' : 'A9', ....}
    return community_values, community_suits

FUNCTION: return array of Player objects with values and suits counts of players' cards

In [ ]:
def get_players_values_suits_dict(encoded_row):
    players_values_suits_arr = []
    players_cards = encoded_row[0:12] # array of encoded players cards
    community_cards = encoded_row[12:18] # array of encoded community cards
    # create dictionary for the row {'P1' : {values}, {suits}, 'P2' : {values}, {suits}}
    community_values_dict, community_suits_dict = get_community_values_suits_dict(community_cards)
    for j in range(0, 12, 2):
        # for each player

        player_values_dict, player_suits_dict = community_values_dict.copy(), community_suits_dict.copy() # fill player values and suits from community cards
        card1 = players_cards[j] # first player's card
        card2 = players_cards[j + 1] # second player's card
        if card1.value != '0' and card2.value != '0':
            # count player values
            if card1.value in player_values_dict:
                player_values_dict[card1.value] += 1
            else:
                player_values_dict[card1.value] = 1
            if card2.value in player_values_dict:
                player_values_dict[card2.value] += 1
            else:
                player_values_dict[card2.value] = 1
            # add values to player's suits + add values to community suits
            if (card1.suit) in player_suits_dict:
                player_suits_dict['p' + card1.suit] += card1.value
                player_suits_dict[card1.suit] += card1.value
 
            if (card2.suit) in player_suits_dict:
                player_suits_dict['p' + card2.suit] += card2.value
                player_suits_dict[card2.suit] += card2.value

            player_values_dict['Kicker1'] = get_value_score(card1.value)[0] # add Kickers
            player_values_dict['Kicker2'] = get_value_score(card2.value)[0]
            
            players_values_suits_arr.append(Player(player_values_dict, player_suits_dict))
        else:
            players_values_suits_arr.append(Player({'Kicker1' : 0, 'Kicker2' : 0}, {'0' : '0'}))
            
    return players_values_suits_arr

FUNCTION: return player combination as Combination object

In [ ]:
def get_player_combination(dict_values, dict_suits, player):
    # save kickers
    kicker1 = dict_values['Kicker1']
    kicker2 = dict_values['Kicker2']
    # delete kickers from dict_values
    dict_values.pop('Kicker1')
    dict_values.pop('Kicker2')

    straight_set = set(dict_values.keys())

    pairs, three_of_kind, four_of_kind = '', '', ''
    flush = [(0, 0)]
    flush_flag, straight_flag, full_house_flag = 0, 0, 0

    for key, value in dict_values.items():
        if value == 2:
            pairs += key
        if value == 3:
            three_of_kind += key
        if value == 4:
            four_of_kind += key

    for key, value in dict_suits.items():
        if len(value) >= 5:
            flush_flag = 1
            suits = dict_suits['p' + key]
            for i in range(len(suits)):
                flush.append((get_value_score(suits[i])[0], key))

    scores_pairs = get_value_score(pairs)         
    str_score = get_straight_score(straight_set)

    #Four of a Kind
    if four_of_kind:
        scores_four_of_kind = get_value_score(four_of_kind)
        combination = Combination('Four of a Kind', 7, scores_four_of_kind[-1], scores_four_of_kind)
    #Full House
    elif pairs and three_of_kind or len(three_of_kind) > 1:
        full_house_flag = 1
        scores_three_of_kind = get_value_score(three_of_kind)   
        pair_full_house = []
        if pairs:
            pair_full_house.append(scores_pairs[-1])
        if len(three_of_kind) > 1:
            pair_full_house.append(scores_three_of_kind[-2])
        combination = Combination('Full House', 6, scores_three_of_kind[-1], max(pair_full_house))
    #Flush
    elif flush_flag:
        flush_score = max(flush)
        combination = Combination('Flush', 5, flush_score[0], flush_score[1])
    #Straight
    elif len(straight_set) >= 5 and str_score != 0:
        straight_flag = 1
        combination = Combination('Straight', 4, str_score, straight_set)
    #Three of Kind
    elif three_of_kind:
        scores_three_of_kind = get_value_score(three_of_kind)
        combination = Combination('Three of a Kind', 3, scores_three_of_kind[-1], scores_three_of_kind)
    #Two Pairs
    elif len(pairs) >= 2:
        combination = Combination('Two Pairs', 2, scores_pairs[-1], scores_pairs[-2])
    #Pair
    elif pairs:
        combination = Combination('Pair', 1, scores_pairs[-1], scores_pairs)
    else:
        combination = Combination('Highest Card', 0, max(kicker1, kicker2), '0')

    if straight_flag or flush_flag or full_house_flag:
        combination.append_kicker(0)
    elif combination.score == kicker1:
        combination.append_kicker(kicker2)
    else:
        combination.append_kicker(kicker1)
    combination.append_player(player)
    return combination

In [ ]:
def get_table_combinations(dict_values, dict_suits):
    combinations = {'Pairs_table' : 0, 'Three of a Kind_table' : 0, 'Straight_table' : 0, 'Flush_table' : '', 'Four of a Kind_table' : 0}
    straight_set = set(dict_values.keys())

    pairs, three_of_kind, four_of_kind = '', '', ''
    flush = []
    flush_flag, straight_flag, full_house_flag = 0, 0, 0

    for key, value in dict_values.items():
        if value == 2:
            pairs += key
        if value == 3:
            three_of_kind += key
        if value == 4:
            four_of_kind += key

    for key, value in dict_suits.items():
        if len(value) >= 5:
            flush_flag = 1
            flush.append(key)

    scores_pairs = get_value_score(pairs)         
    str_scores = get_straight_scores(straight_set)
    scores_three_of_kind = get_value_score(three_of_kind)   

    #Four of a Kind
    if four_of_kind:
        scores_four_of_kind = get_value_score(four_of_kind)
        combinations['Four of a Kind_table'] = scores_four_of_kind
    #Flush
    if flush_flag:
        combinations['Flush_table'] = flush
    #Straight
    if len(straight_set) >= 5 and str_scores != []:
        combinations['Straight_table'] = str_scores
    #Three of Kind
    if three_of_kind:
        combinations['Three of a Kind_table'] = scores_three_of_kind
    #Pair
    if pairs:
        combinations['Pairs_table'] = scores_pairs

    return combinations

FUNCTION: return winners as an array of Combination object

In [ ]:
def get_winners(combinations):

    max_combination, max_score, max_kicker, max_pair, max_kicker1, max_kicker2 = 0, 0, 0, 0, 0, 0
    combination_winners, score_winners, second_pair, winners, kicker1_winners = [], [], [], [], []
    # get score of max combination
    for i in range(len(combinations)):
        if combinations[i].combination_score >= max_combination:
            max_combination = combinations[i].combination_score
            winning_combination = combinations[i].combination_name
    # get list of players with max combination
    for i in range(len(combinations)):
        if combinations[i].combination_score == max_combination:
            combination_winners.append(combinations[i])
            
    if len(combination_winners) > 1:
        # get max score of combination
        for i in range(len(combination_winners)):
            if combination_winners[i].score > max_score:
                max_score = combination_winners[i].score
        # get list of players with max combination
        for i in range(len(combination_winners)):
            if combination_winners[i].score == max_score:
                score_winners.append(combination_winners[i])

        if len(score_winners) > 1:
            if (winning_combination != 'Full House') & (winning_combination != 'Two Pairs'):
                for i in range(len(score_winners)):
                    if score_winners[i].kicker > max_kicker:
                        max_kicker = score_winners[i].kicker
                for i in range(len(score_winners)):
                    if score_winners[i].kicker == max_kicker:
                        winners.append(score_winners[i])
            else:
                for i in range(len(score_winners)):
                    if score_winners[i].bonus > max_pair:
                        max_pair = score_winners[i].bonus
                for i in range(len(score_winners)):
                    if score_winners[i].bonus == max_pair:
                        second_pair.append(score_winners[i])
                if len(second_pair) > 1:
                    for i in range(len(second_pair)):
                        if second_pair[i].kicker > max_kicker:
                            max_kicker = second_pair[i].kicker
                    for i in range(len(second_pair)):    
                        if second_pair[i].kicker == max_kicker:
                            winners.append(second_pair[i])
                else:
                    winners = second_pair
        else:
            winners = score_winners
    else:
        winners = combination_winners  
    return winners

FUNCTION: create dataframe with winning combination and winner

In [ ]:
def create_winner_df(encoded_arr):
    df = pd.DataFrame(columns = ['P1_COMB', 'P1_SCORE', 'P1_BONUS', 'KICKER_P1', 'P2_COMB', 'P2_SCORE', 'P2_BONUS', 'KICKER_P2', 'P3_COMB', 'P3_SCORE', 'P3_BONUS',
                                 'KICKER_P3', 'P4_COMB', 'P4_SCORE', 'P4_BONUS',  'KICKER_P4', 'P5_COMB', 'P5_SCORE', 'P5_BONUS', 'KICKER_P5', 'P6_COMB', 
                                 'P6_SCORE', 'P6_BONUS', 'KICKER_P6', 'WINNING COMBINATION', 'WINNER SCORE', 'WINNER', 'BONUS'])
    for i in range(len(encoded_arr)):

        row, combinations, win_players = [], [], []
        table = get_players_values_suits_dict(encoded_arr[i])
        
        for j in range(len(table)):
            combination = get_player_combination(table[j].values, table[j].suits, 'P' + str(j + 1))
            row.append(combination.combination_name)
            row.append(combination.score)
            row.append(combination.bonus)
            row.append(combination.kicker)
            combinations.append(combination)
        
        winners = get_winners(combinations)
        row.append(winners[0].combination_name)
        row.append(winners[0].score)
        if len(winners) > 1:
            for k in range(len(winners)):
                win_players.append(winners[k].player)
            row.append(win_players)
            row.append(winners[0].bonus)
        else:
            row.append(winners[0].player)
            row.append(winners[0].bonus)
        df.loc[i] = row
    return df

In [ ]:
def clean_arr(arr):
    new_arr = []
    s = list(set(arr))
    for i in range(len(s)):
        if s[i] != 0:
            new_arr.append(s[i])
    return new_arr

In [ ]:
def create_combinations_df(encoded_arr):
    df = pd.DataFrame()
    df_players_comb = pd.DataFrame()
    dicts = []
    dicts_players = []
    for i in range(len(encoded_arr)):
        players_dict = {}
        com_vals, com_suits = get_community_values_suits_dict(encoded_arr[i])
        combinations = get_table_combinations(com_vals, com_suits)
        table = get_players_values_suits_dict(encoded_arr[i])
        for j in range(len(table)):
            combination = get_player_combination(table[j].values, table[j].suits, 'P' + str(j + 1))
            if combination.combination_name in players_dict:
                arr = players_dict[combination.combination_name]
                if combination.combination_name == 'Flush':
                    arr.append(combination.bonus)
                else:
                    arr.append(combination.score)
                players_dict[combination.combination_name] = clean_arr(arr)
            else:
                if combination.combination_name == 'Flush':
                     players_dict[combination.combination_name] = [combination.bonus]
                else:
                    players_dict[combination.combination_name] = [combination.score]
        dicts_players.append(players_dict)
        dicts.append(combinations)
    df = df.append(dicts, ignore_index=True, sort=False)
    df_players_comb = df_players_comb.append(dicts_players, ignore_index=True, sort=False)
    return df, df_players_comb

In [ ]:
def create_pl_combinations_df(encoded_arr):
    df = pd.DataFrame()
    dicts = []
    for i in range(len(encoded_arr)):
        com_vals, com_suits = get_community_values_suits_dict(encoded_arr[i])
        combinations = get_table_combinations(com_vals, com_suits)
        dicts.append(combinations)
    df = df.append(dicts, ignore_index=True, sort=False)
    return df

FUNCTION: correct winner players for full dataframe 

In [ ]:
def correct_players_full(win_df_full):
    win_df_full.iloc[365, 20] = 'P2'
    win_df_full.iloc[486, 20] = 'P2'
    win_df_full.iloc[527, 20] = 'P5'
    win_df_full.iloc[780, 20] = 'P3'
    win_df_full.iloc[848, 20] = 'P1'
    win_df_full.iloc[897, 20] = 'P5'
    win_df_full.iloc[974, 20] = 'P4'
    win_df_full.iloc[1248, 20] = 'P3'
    win_df_full.iloc[1291, 20] = 'P2P5'
    win_df_full.iloc[1640, 20] = 'P3'
    win_df_full.iloc[1807, 20] = 'P6'
    win_df_full.iloc[2023, 20] = 'P3'
    win_df_full.iloc[2065, 20] = 'P4'
    return win_df_full

FUNCTION: check occasional duplicates in dataframe

In [ ]:
def check_dublicates(df_tables_full):
    for i in range(len(df_tables_full)):
        if (len(df_tables_full.loc[i]) - list(df_tables_full.loc[i]).count('0') - len(np.array(np.unique(df_tables_full.loc[i].values)))) > 0:
            print(i, df_tables_full.loc[i])

FUNCTION: ind rows with players mistakes 

In [ ]:
def find_winner_errors(win_df_full):
    errors_df = win_df_full[(win_df_full['WINNER'] != 'P1')  & (win_df_full['WINNER'] != 'P2') & (win_df_full['WINNER'] != 'P3') & (win_df_full['WINNER'] != 'P4') & (win_df_full['WINNER'] != 'P5') & (win_df_full['WINNER'] != 'P6')]
    errors_df_full = errors_df[(errors_df['WINNING COMBINATION'] == 'Pair') | (errors_df['WINNING COMBINATION'] == 'Two Pairs')]
    return errors_df_full

In [ ]:
def count_cards_num(arr):
    count = 0
    for i in range(len(arr)):
        if arr[i] != '0':
            count += 1
    return count 

# DATA 

Create dataframe with cards only - df_tables_full

 Make slices for presened flop only, turn, river, no community cards and all players 

In [ ]:
df_tables_full = data

df_tables_river = df_tables_full[df_tables_full.River != '0']
df_tables_turn = df_tables_full[(df_tables_full.Turn  != '0') & (df_tables_full.River == '0')]
df_tables_flop = df_tables_full[(df_tables_full.Flop1 != '0') & (df_tables_full.Turn  == '0')]
df_tables_community = df_tables_full[(df_tables_full.Flop1 != '0')]
df_tables_no_community = df_tables_full[df_tables_full.Flop1 == '0']
df_tables_all_players = df_tables_full[(df_tables_full.P1H2 != '0') & (df_tables_full.P2H2 != '0') & (df_tables_full.P3H2 != '0') & (df_tables_full.P4H2 != '0') & (df_tables_full.P5H2 != '0') & (df_tables_full.P6H2 != '0')]
df_tables_all_players_no_community = df_tables_all_players[df_tables_all_players.Flop1 == '0']

In [ ]:
arr_counts = []
for i in range(len(df_tables_full)):
    arr_counts.append(count_cards_num(df_tables_full.loc[i]))

Encode dataframe slices to 2D array 

In [ ]:
encoded_mtrx_full = encode_df_cards(df_tables_full)
encoded_mtrx_river = encode_df_cards(df_tables_river)
encoded_mtrx_turn = encode_df_cards(df_tables_turn)
encoded_mtrx_flop = encode_df_cards(df_tables_flop)
encoded_mtrx_community = encode_df_cards(df_tables_community)
encoded_mtrx_no_community = encode_df_cards(df_tables_no_community)
encoded_mtrx_all_players = encode_df_cards(df_tables_all_players)
encoded_mtrx_all_players_no_community = encode_df_cards(df_tables_all_players_no_community)

# Tests

In [ ]:
pd.set_option('display.max_rows', 2100)
pd.set_option('display.max_columns', 50)

# All players corrected df

In [ ]:
win_df_full = create_winner_df(encoded_mtrx_full)
win_df_flop = create_winner_df(encoded_mtrx_flop)
win_df_turn = create_winner_df(encoded_mtrx_turn)
win_df_river = create_winner_df(encoded_mtrx_river)
win_df_community = create_winner_df(encoded_mtrx_community)
win_df_all_players = create_winner_df(encoded_mtrx_all_players)
win_df_no_community = create_winner_df(encoded_mtrx_no_community)
win_df_all_players_no_community = create_winner_df(encoded_mtrx_all_players_no_community)

In [ ]:
df_player1_combinations = create_pl_combinations_df(df_player1)
df_player2_combinations = create_pl_combinations_df(df_player2)
df_player3_combinations = create_pl_combinations_df(df_player3)
df_player4_combinations = create_pl_combinations_df(df_player4)
df_player5_combinations = create_pl_combinations_df(df_player5)
df_player6_combinations = create_pl_combinations_df(df_player6)

In [ ]:
flop_flag = []
for i in range(1, len(data)):
    if data.iloc[i, 12] != '0':
        flop_flag.append(1)
    else:
        flop_flag.append(0)

In [ ]:
df_counts = pd.DataFrame(index=[2,3,	4,	5,	6,	7,	8,	9,	10,	11,12,13,14], columns = ['Three of a Kind', 'Full House', 'Four of a Kind'])
df_counts['Pair'] = [243,	246,	264,	223	,238,	278	,276	,233,	265	,249	,265	,290	,298]
df_counts['Two Pairs'] = [0,	13	,38	,43	,80	,94	,117,	95,	141	,155	,168	,262,277]
df_counts['Three of a Kind'] = [26	,23	,22,	28	,21,	35	,40,	17,	24,	25,	23,	37,	32]
df_counts['Full House'] = [8	,9	,10	,27,	8,	20,	27,	11	,12	,12,	9,	17	,18]
df_counts['Four of a Kind'] = [0	,0	,0	,3,	0,	0	,4	,0	,3,	1	,3	,3	,1]
# sns.barplot(x = df_counts.index, y = df_counts['Three of a Kind'])
sns.barplot(x = df_counts.index, y = df_counts['Three of a Kind'])
plt.show()

In [ ]:
df_player1_combinations['players comb'] = win_df_full.iloc[1:, 0].reset_index(drop=True)
df_player1_combinations['score'] = win_df_full.iloc[1:, 1].reset_index(drop=True)
df_player1_combinations['bonus'] = win_df_full.iloc[1:, 2].reset_index(drop=True)
df_player1_combinations['flop flag'] = flop_flag
df_player1_combinations.replace('', '0', inplace=True)
# df_player1_combinations[df_player1_combinations['players comb'] == 'Flush']

df_player2_combinations['players comb'] = win_df_full.iloc[1:, 4].reset_index(drop=True)
df_player2_combinations['score'] = win_df_full.iloc[1:, 5].reset_index(drop=True)
df_player2_combinations['bonus'] = win_df_full.iloc[1:, 6].reset_index(drop=True)
df_player2_combinations['flop flag'] = flop_flag
df_player2_combinations.replace('', '0', inplace=True)

df_player3_combinations['players comb'] = win_df_full.iloc[1:, 8].reset_index(drop=True)
df_player3_combinations['score'] = win_df_full.iloc[1:, 9].reset_index(drop=True)
df_player3_combinations['bonus'] = win_df_full.iloc[1:, 10].reset_index(drop=True)
df_player3_combinations['flop flag'] = flop_flag
df_player3_combinations.replace('', '0', inplace=True)

df_player4_combinations['players comb'] = win_df_full.iloc[1:, 12].reset_index(drop=True)
df_player4_combinations['score'] = win_df_full.iloc[1:, 13].reset_index(drop=True)
df_player4_combinations['bonus'] = win_df_full.iloc[1:, 14].reset_index(drop=True)
df_player4_combinations['flop flag'] = flop_flag
df_player4_combinations.replace('', '0', inplace=True)

df_player5_combinations['players comb'] = win_df_full.iloc[1:, 16].reset_index(drop=True)
df_player5_combinations['score'] = win_df_full.iloc[1:, 17].reset_index(drop=True)
df_player5_combinations['bonus'] = win_df_full.iloc[1:, 18].reset_index(drop=True)
df_player5_combinations['flop flag'] = flop_flag
df_player5_combinations.replace('', '0', inplace=True)

df_player6_combinations['players comb'] = win_df_full.iloc[1:, 20].reset_index(drop=True)
df_player6_combinations['score'] = win_df_full.iloc[1:, 21].reset_index(drop=True)
df_player6_combinations['bonus'] = win_df_full.iloc[1:, 22].reset_index(drop=True)
df_player6_combinations['flop flag'] = flop_flag
df_player6_combinations.replace('', '0', inplace=True)

In [ ]:
df = pd.concat([df_player1_combinations, df_player2_combinations, df_player3_combinations, df_player4_combinations, df_player5_combinations, df_player6_combinations]).reset_index(drop= True)

In [ ]:
def straight_df(df):  
    df_straight = pd.DataFrame(columns = ['score_5', 'score_6', 'score_7', 'score_8', 'score_9', 'score_10', 'score_11', 'score_12', 'score_13', 'score_14'])
    for i in range(len(df)):
        arr_scores = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        arr = df.iloc[i, 2]
        if arr != 0:
            for j in range(len(arr)):
                if arr[j] == 5:
                    arr_scores[0] = 1
                if arr[j] == 6:
                    arr_scores[1] = 1
                if arr[j] == 7:
                    arr_scores[2] = 1
                if arr[j] == 8:
                    arr_scores[3] = 1
                if arr[j] == 9:
                    arr_scores[4] = 1
                if arr[j] == 10:
                    arr_scores[5] = 1
                if arr[j] == 11:
                    arr_scores[6] = 1
                if arr[j] == 12:
                    arr_scores[7] = 1
                if arr[j] == 13:
                    arr_scores[8] = 1
                if arr[j] == 14:
                    arr_scores[9] = 1
        # print(arr_scores)
        df_straight.loc[i] = arr_scores
    df_straight = df_straight[df['flop flag'] == 1]
    return (df_straight)

In [ ]:
straight = df[df['players comb'] == 'Straight']

In [ ]:
df = straight_df(df)

In [ ]:
df_XY = pd.merge(df, straight['score'], left_index = True, right_index=True, how = 'left').fillna(0)

In [ ]:
X = df_XY.iloc[:, :10]
y = df_XY['score']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['score']
print(classification_report(y_test, y_pred))

In [ ]:
def matches(df_flush, pred, real):
    flush_matches = []
    for i in range(len(df_flush)):
        if df_flush.iloc[i, real] != 0 and df_flush.iloc[i, pred] != 0:
            if df_flush.iloc[i, real] in df_flush.iloc[i, pred]:
                flush_matches.append(1)
            else:
                flush_matches.append(-1)
        else:
            flush_matches.append(0)
    df_flush['match'] = flush_matches
    unique, counts_flush = np.unique(flush_matches, return_counts=True)
    return unique, counts_flush

In [ ]:
matches(df_player1_combinations, 2, 6)

In [ ]:
df_combination, df_players = create_combinations_df(encoded_mtrx_full)
df_players = df_players.fillna(0)
df_players.loc[len(df_players)] = df_players.loc[len(df_players) - 1]
df_players = df_players[1:].reset_index(drop = True)
df_players = df_players.fillna(0)
df_combinations = pd.concat([df_combination, df_players], axis = 1)
df_combinations = df_combinations.reindex(columns=['Pair', 'Two Pairs', 'Pairs_table', 'Three of a Kind', 'Three of a Kind_table', 'Straight', 'Straight_table', 'Flush', 'Flush_table', 
                                                                              'Four of a Kind', 'Four of a Kind_table', 'Full House'])
df_combinations['cards count'] = arr_counts

In [ ]:
win_df_full[win_df_full['WINNING COMBINATION'] == 'Straight']

In [ ]:
df_combinations

In [ ]:
df_flush = df_combinations[df_combinations.Flush != 0]
df_pair = df_combinations[df_combinations.Pair != 0]
df_pair = df_pair[['Pair', 'Pairs_table']]

In [ ]:
def match(df):   
    matches = []
    for i in range(len(df)):
        if df.iloc[i, 1] != 0:
            if df.iloc[i, 0] in df.iloc[i, 1]:
                matches.append(1)
            else:
                matches.append(-1)
        else:
            matches.append(0)
    df['match'] = matches
    return df
    # unique, counts_flush = np.unique(matches, return_counts=True)

In [ ]:
flush_matches = []
for i in range(len(df_flush)):
    if df_flush.iloc[i, 7] != 0:
        if df_flush.iloc[i, 7][0] in df_flush.iloc[i, 8]:
            flush_matches.append(1)
        else:
            flush_matches.append(-1)
    else:
        flush_matches.append(0)
df_flush['match'] = flush_matches
unique, counts_flush = np.unique(flush_matches, return_counts=True)

In [ ]:
straight_matches = []
for i in range(len(df_combinations)):
    if df_combinations.iloc[i, 5] != 0 and df_combinations.iloc[i, 6] != 0:
        if df_combinations.iloc[i, 5][0] in df_combinations.iloc[i, 6]:
            straight_matches.append(1)
        else:
            straight_matches.append(-1)
    else:
        straight_matches.append(0)
unique, counts_straight = np.unique(straight_matches, return_counts=True)

In [ ]:
list_df_slices = list()
list_df_slices.append(win_df_full[0:48])

In [ ]:
for i in range(len(games_indexes) - 1):
    df = win_df_full.loc[games_indexes[i]:games_indexes[i+1]]
    list_df_slices.append(df)

In [ ]:
errors_all_players = find_winner_errors(win_df_all_players)
errors_all_players

In [ ]:
win_df_all_players.iloc[264, 20] = 'P3'
win_df_all_players.iloc[311, 20] = 'P4'

In [ ]:
without_highest_card = win_df_all_players[win_df_all_players['WINNING COMBINATION'] != 'Highest Card']

In [ ]:
df_karmanka = win_df_all_players_no_community[win_df_all_players_no_community['WINNING COMBINATION'] == 'Pair']
df_karmanka['WINNER'].value_counts()

In [ ]:
df_highest_card = win_df_all_players_no_community[win_df_all_players_no_community['WINNING COMBINATION'] == 'Highest Card']
df_highest_card['WINNER'].value_counts()

In [ ]:
da = pd.DataFrame({'Total Variation distance': [1.00000	,1.00000	,1.00000,	0.99999,	0.92414,	0.61411,	0.33551,	0.16755,	0.08517,	0.04258], 
                   'Threshold' : [0.33551, 0.33551, 0.33551, 0.33551, 0.33551, 0.33551, 0.33551, 0.33551, 0.33551, 0.33551]}, index = ['1 riffle', '2 riffles', '3 riffles', '4 riffles', '5 riffles', '6 riffles',
                                                                                                                                       '7 riffles', '8 riffles', '9 riffles', '10 riffles'])
sns.lineplot(data = da)
plt.tight_layout()

In [ ]:
p1 = win_df_community[win_df_community['P1_SCORE'] != 0].P1_COMB.value_counts().fillna(0)
p2 = win_df_community[win_df_community['P2_SCORE'] != 0].P2_COMB.value_counts().fillna(0)
p3 = win_df_community[win_df_community['P3_SCORE'] != 0].P3_COMB.value_counts().fillna(0)
p4 = win_df_community[win_df_community['P4_SCORE'] != 0].P4_COMB.value_counts().fillna(0)
p5 = win_df_community[win_df_community['P5_SCORE'] != 0].P5_COMB.value_counts().fillna(0)
p6 = win_df_community[win_df_community['P6_SCORE'] != 0].P6_COMB.value_counts().fillna(0)

In [ ]:
win_df_community.head()

In [ ]:
d1 = win_df_community.iloc[:, :2].value_counts().to_frame('counts')
d2 = win_df_community.iloc[:, 4:6].value_counts().to_frame('counts')
d3 = win_df_community.iloc[:, 8:10].value_counts().to_frame('counts')
d4 = win_df_community.iloc[:, 12:14].value_counts().to_frame('counts')
d5 = win_df_community.iloc[:, 16:18].value_counts().to_frame('counts')
d6 = win_df_community.iloc[:, 20:22].value_counts().to_frame('counts')

In [ ]:
c = pd.concat([d1, d2, d3, d4, d5, d6], axis = 1).sum(axis = 1)
c

In [ ]:
comb_counts = np.array([3368, 1965, 1483, 353, 237, 236, 188, 18, 0, 0] )
sum(comb_counts)

In [ ]:
plt.figure(figsize=(10, 6))
# comb_counts = [3368, 1483, 353, 237, 236, 188, 18, 0, 0]
ax = sns.barplot(x = ['Pair', 'High Card', 'Two Pairs', 'Three of a Kind', 'Straight', 'Flush', 'Full House', 'Four of a Kind', 'Straight Flush', 'Royal Flush'], y = comb_counts / 7848 * 100, palette = 'rocket_r')
plt.ylabel('Probability')
ax.bar_label(ax.containers[0])
plt.tight_layout()

# Winner distribution graph

In [ ]:
win_counts = [110, 102, 101, 100, 96, 92]
win_players = ['P3', 'P1', 'P2', 'P6', 'P5', 'P4']
plt.bar(win_players, win_counts, color = ['rebeccapurple', 'darkslateblue', 'slateblue', 'mediumslateblue', 'mediumpurple', 'plum'])
plt.title('WINNER DISTRIBUTION')

In [ ]:
win_counts = [97, 87, 86, 78, 77, 77]
win_players = ['P3', 'P6', 'P2', 'P1', 'P4', 'P5']
plt.bar(win_players, win_counts, color = ['rebeccapurple', 'darkslateblue', 'slateblue', 'mediumslateblue', 'mediumpurple', 'plum'])
plt.title('WINNER DISTRIBUTION WITHOUT HIGHEST CARD')

In [ ]:
win_counts = [33, 27, 27, 27, 25, 23]
win_players = ['P1', 'P3', 'P5', 'P4', 'P2', 'P6']
plt.bar(win_players, win_counts, color = ['rebeccapurple', 'darkslateblue', 'slateblue', 'mediumslateblue', 'mediumpurple', 'plum'])
plt.title('WINNER DISTRIBUTION WITHOUT COMMUNITY CARDS')

In [ ]:
win_counts = [14, 13, 12, 10, 10, 9]
win_players = ['P3', 'P4', 'P2', 'P5', 'P1', 'P6']
plt.bar(win_players, win_counts, color = ['rebeccapurple', 'darkslateblue', 'slateblue', 'mediumslateblue', 'mediumpurple', 'plum'])
plt.title('KARMANKA DISTRIBUTION')

In [ ]:
win_counts = [23, 17, 14, 14, 14, 13]
win_players = ['P1', 'P5', 'P4', 'P6', 'P2', 'P3']
plt.bar(win_players, win_counts, color = ['rebeccapurple', 'darkslateblue', 'slateblue', 'mediumslateblue', 'mediumpurple', 'plum'])
plt.title('HIGHEST CARD DISTRIBUTION')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8,6))
# fig = plt.figure(figsize=(7, 6))
sns.barplot(x = win_df_river['WINNING COMBINATION'].value_counts().index, y =win_df_river['WINNING COMBINATION'].value_counts() / len(win_df_river), palette="rocket_r")
plt.ylabel('Probability')
plt.title('RIVER')
plt.xlabel('')
# sns.countplot(ax = axes[0], data=win_df_flop, x='WINNING COMBINATION', palette="rocket_r", order=win_df_flop['WINNING COMBINATION'].value_counts().index)
# axes[0].set_title('FLOP')
# sns.countplot(ax = axes[1], data=win_df_turn, x='WINNING COMBINATION', palette="rocket_r", order=win_df_turn['WINNING COMBINATION'].value_counts().index)
# axes[1].set_title('TURN')
# sns.countplot(ax = axes[2], data=win_df_river, x='WINNING COMBINATION', palette="rocket_r", order=win_df_river['WINNING COMBINATION'].value_counts().index)
# axes[2].set_title('RIVER')
plt.tight_layout()

In [ ]:
P2 = 741 / 2538
P1 = 720 / 2538
P3 = 501 / (2538 - 375)
P4 = 325 / (2538 - 925)
P5 = 199 / (2538 - 1420)
P6 = 121 / (2538 - 1850)
players_counts = [P2, P1, P3, P4, P5, P6]

In [ ]:
fig = plt.figure(figsize=(7, 6))
sns.barplot(x = ['1', '2', '3', '4', '5', '6'],y = players_counts, palette="viridis_r")
plt.ylabel('Probability of winning')
plt.xlabel('Position')

# Cards distribution by player graph

df_card_stats - dataframe of each card occurence for each player, flop, turn and river

In [ ]:
def df_stats(df_tables_full):
    df_card_count = pd.concat([df_tables_full['P1H1'].value_counts(), df_tables_full['P1H2'].value_counts(), df_tables_full['P2H1'].value_counts(), df_tables_full['P2H2'].value_counts(),
                            df_tables_full['P3H1'].value_counts(), df_tables_full['P3H2'].value_counts(), df_tables_full['P4H1'].value_counts(), df_tables_full['P4H2'].value_counts(),
                            df_tables_full['P5H1'].value_counts(), df_tables_full['P5H2'].value_counts(), df_tables_full['P6H1'].value_counts(), df_tables_full['P6H2'].value_counts(),
                            df_tables_full['Flop1'].value_counts(),df_tables_full['Flop2'].value_counts(),df_tables_full['Flop3'].value_counts(), df_tables_full['Turn'].value_counts(), df_tables_full['River'].value_counts()], axis = 1)
    df_card_count = df_card_count.fillna(0)

    df_concat = pd.concat([df_card_count['P1H1'] + df_card_count['P1H2'], df_card_count['P2H1'] + df_card_count['P2H2'], df_card_count['P3H1'] + df_card_count['P3H2'],
                        df_card_count['P4H1'] + df_card_count['P4H2'], df_card_count['P5H1'] + df_card_count['P5H2'], df_card_count['P6H1'] + df_card_count['P6H2'],
                        df_card_count['Flop1'],  df_card_count['Flop2'], df_card_count['Flop3'], df_card_count['Turn'], df_card_count['River']], axis = 1)
    df_card_stats = pd.DataFrame(df_concat, index=['AH', 'AC', 'AD', 'AS', 'KH', 'KC', 'KD', 'KS', 'QH', 'QC', 'QD', 'QS', 'JH', 'JC', 'JD', 'JS', 'TH', 'TC', 'TD', 'TS', 
                                                    '9H', '9C', '9D', '9S', '8H', '8C', '8D', '8S', '7H', '7C', '7D', '7S', '6H', '6C', '6D', '6S', '5H', '5C', '5D', '5S',
                                                    '4H', '4C', '4D', '4S', '3H', '3C', '3D', '3S', '2H', '2C', '2D', '2S'])
    df_card_stats = df_card_stats.set_axis(['PLAYER1', 'PLAYER2', 'PLAYER3', 'PLAYER4', 'PLAYER5', 'PLAYER6', 'FLOP1', 'FLOP2', 'FLOP3', 'TURN', 'RIVER'], axis=1, inplace=False)
    return df_card_stats

In [ ]:
df_card_stats = df_stats(df_tables_all_players)

Colors for ranks by groups: (A, K, Q, J, T), (9, 8, 7, 6) (5, 4, 3, 2)

In [ ]:
deck_values_colors = ['indigo', 'indigo', 'indigo', 'indigo', 'blueviolet', 'blueviolet', 'blueviolet', 'blueviolet', 'slateblue', 'slateblue', 'slateblue',
       'slateblue', 'mediumpurple', 'mediumpurple', 'mediumpurple', 'mediumpurple', 'plum', 'plum', 'plum', 'plum', 'aqua', 'aqua',
       'aqua', 'aqua', 'turquoise', 'turquoise', 'turquoise', 'turquoise', 'mediumturquoise', 'mediumturquoise', 'mediumturquoise',
       'mediumturquoise', 'lightseagreen', 'lightseagreen', 'lightseagreen', 'lightseagreen', 'khaki', 'khaki', 'khaki',
       'khaki', 'lemonchiffon', 'lemonchiffon', 'lemonchiffon', 'lemonchiffon', 'palegoldenrod', 'palegoldenrod', 'palegoldenrod',
       'palegoldenrod', 'darkkhaki', 'darkkhaki', 'darkkhaki','darkkhaki']

In [ ]:
def cards_distribution_graph(df_card_stats):    
    fig = plt.figure(figsize=(25, 30))
    ax1 = fig.add_subplot(11, 1, 1)
    ax1.bar(df_card_stats.index, df_card_stats.iloc[:, 0], color = deck_values_colors)
    ax1.set_title(df_card_stats.columns[0])
    for i in range(1, df_card_stats.shape[1]):
        ax = fig.add_subplot(11, 1, i + 1, sharey = ax1)
        ax.bar(df_card_stats.index, df_card_stats.iloc[:, i], color = deck_values_colors)
        ax.set_title(df_card_stats.columns[i])
    plt.tight_layout()

In [ ]:
cards_distribution_graph(df_card_stats)